In [43]:
import pandas as pd
from functools import reduce
import numpy as np
import os
from fuzzywuzzy import fuzz, process
from datetime import datetime


In [44]:
# 创建df1
data1 = {
    'Name': ['Tom', 'Nick', 'John', 'Peter'],
    'Age': [20, 21, 19, 18],
    'Subject': ['Math', 'Physics', 'Chemistry', 'Biology'],
    'Score': [80, 90, 78, 88]
}
df1 = pd.DataFrame(data1)
df1 = df1.astype(str)
# 创建df2_clean_clean_clean_clean_clean_clean_clean，它包含与df1类似但带有拼写错误或不准确的姓名、学科
data2 = {
    'Name': ['tome', 'Nic', 'Johne', 'Pete'],
    'Age': [20, 21, 19, 18],
    'Subject': ['Maths', 'Physic', 'Chem', 'Bio'],
    'Score': [80.00, 90, 78, 88]
}
df2 = pd.DataFrame(data2)


In [45]:
def clean_dataframe(df, capitalize_columns=[],capitalize_name=True):

    df_clean = df.copy()
    # 将所有列转换为字符串类型
    df_clean = df_clean.astype(str)

    # 移除所有列中的空格
    for col in df.columns:
        df_clean[col] = df_clean[col].str.replace(' ', '')
    
    # 将指定的列的首字母转换为大写
    if capitalize_name:
        for col in capitalize_columns:
            if col in df.columns:
                df_clean[col] = df_clean[col].str.title()
                # df_clean[col] = df_clean[col].str.upper()

    return df_clean

# df2_clean=df2[[ '标准',]].drop_duplicates()
df2_clean = clean_dataframe(df2, capitalize_columns=['Name'])
df2_clean

,Name,Age,Subject,Score
0,Tome,20,Maths,80.0
1,Nic,21,Physic,90.0
2,Johne,19,Chem,78.0
3,Pete,18,Bio,88.0


In [46]:
output_lines = []
output_lines.append("标准姓名列")
changed_cells = []

name_col_num = df2_clean.columns.get_loc('Name')

# 遍历df2_clean的每一行
for index, row in df2_clean.iterrows():
    if row['Name'] not in df1['Name'].unique():
        closest_matches = process.extract(row['Name'], df1['Name'].unique(), limit=3)
        original_value = row['Name']
        replaced_value = closest_matches[0][0]
        
        # 如果最接近的匹配的得分大于等于90，那么就替换相应的值
        if closest_matches[0][1] >= 50:
            df2_clean.at[index, 'Name'] = replaced_value
            changed_cells.append((index, name_col_num))
            output_lines.append(f"原始值: {original_value}, 替换值: {replaced_value} 待选列表：")
            for i, match in enumerate(closest_matches):
                output_lines.append(f"{i+1}: {match[0]}")
            output_lines.append(" ")
        elif closest_matches[0][1] <= 50:
            changed_cells.append((index, name_col_num))
            output_lines.append(f"原始值: {original_value}, 在之前的数据库中未找到.")
            output_lines.append(" ")




In [47]:
def match_Subject(df1, df2, df1_column, df2_column, df1_match1, df1_match2):
    """
    df1:待清洗dataframe
    df1_match1, df1_match2：模糊匹配的列
    df1_column, df2_column：需匹配的列
    df2：标准dataframe
    df1_match1, df1_match2
    """
    output_lines.append(df2_column+"列")
    changed_cells_col_num = df1.columns.get_loc(df1_column)
    for index, row in df1.iterrows():
        if row[df1_column] not in df2[df2_column].unique():
            # matching_rows = df2[(df2['标准'] == row[df1_match1]) & (df2['标准'] == row[df1_match2])]
            matching_rows = df2[(df2[df1_match1] == row[df1_match1]) & (df2[df1_match2] == row[df1_match2])]
            # 使用fuzz.ratio找出df2中最像的三个值
            closest_matches = process.extract(row[df1_column], matching_rows[df2_column].unique(), limit=3)
            output_lines.append(f"原始值 '{row[df1_column]}' 需要替换:")
            if not closest_matches:  # 添加这行代码来判断closest_matches是否为空
                output_lines.append(f"Value '{row[df1_column]}' 但在原数据库中未找到对应的值")
                output_lines.append(" ")
                changed_cells.append((index, changed_cells_col_num))
                continue
            replaced_value = closest_matches[0][0]
            if closest_matches[0][1] >= 90:
                output_lines.append(f"原始值: {row[df1_column]}, 替换值: {replaced_value}")
                df1.at[index, df1_column] = replaced_value
                changed_cells.append((index, changed_cells_col_num))
                for i, match in enumerate(closest_matches):           
                    #print("原始表格" + tabulate(pd.DataFrame(row).T, headers='keys', tablefmt='psql', showindex=False))
                    # output_lines.append(f"Value '{row[df1_column]}' 不在原数据库中，需要替换:")
                    output_lines.append(f"{i+1}: {match[0]}")
                    match_row = matching_rows.loc[matching_rows[df2_column] == match[0]]
                output_lines.append(match_row.to_string(index=False))
                output_lines.append(" ")
            elif closest_matches[0][1] <= 90:
                changed_cells.append((index, changed_cells_col_num))
                output_lines.append(f"原始值: {row[df1_column]}, 在之前的数据库中未找到.")
                output_lines.append(" ")
                

    return df1
df2_clean=match_Subject(df2_clean, df1, df1_column='Subject', df2_column='Subject',df1_match1='Name', df1_match2='Age')

In [48]:
changed_cells_red = []
def mark_unmatched_cells(changed_cells_red,df1, df2_clean,col,df1_match1,df1_match2):
    Form_col_num = df2_clean.columns.get_loc(col)
    for index, row in df2_clean.iterrows():
        matching_rows = df1[(df1[df1_match1] == row[df1_match1]) & (df1[df1_match2] == row[df1_match2])]
        if row[col] not in matching_rows[col].unique():
            changed_cells_red.append((index, Form_col_num))
    return changed_cells_red


In [49]:
file = open('修改日志.txt', 'w')

# 循环遍历列表并写入文件
for item in output_lines:
    file.write("%s\n" % item)

# 关闭文件
file.close()

In [50]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill

# 创建新的工作簿
wb = Workbook()

# 选择活动工作表
sheet = wb.active

# 定义填充颜色
fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type = "solid")
for cell in changed_cells :
    sheet.cell(row=cell[0]+2, column=cell[1]+1).fill = fill

fill = PatternFill(start_color="FFB6C1", end_color="FFB6C1", fill_type="solid")
for cell in changed_cells_red  :
    sheet.cell(row=cell[0]+2, column=cell[1]+1).fill = fill


# 保存到文件
wb.save("colored_cells.xlsx")


In [ ]:
def matched_df()：

    # 初始化一个空的DataFrame来存储找到的行
    matched_df = pd.DataFrame()
    highest_ratio_df = pd.DataFrame()  # 新建一个DataFrame用于存储每个min1对应的fuzz_ratio最高的行

    for index, row in df2_clean.iterrows():

        #筛选防止数据过多
        matching_rows = df1[(df1['Name'] == row['Name']) ]
        a_val = row['1']
        b_val = row['2']
        c_val = row['3']
        d_val = row['4']

        max_ratio = -1  # 初始化一个变量用于记录最大的fuzz_ratio
        max_ratio_row = None  # 初始化一个变量用于记录fuzz_ratio最大的行
        temp_df = pd.DataFrame()
        # 创建一个新的DataFrame来保存本次循环的数据
        isStandardProductionCompanyExists = False
        for idx, r in matching_rows.iterrows():
            if r['标准学科'] == row["学科"]:
                r['标准学科匹配'] = 1
                isStandardProductionCompanyExists = True

            fuzz_value =(fuzz.ratio(r['标准名1'], a_val) + \
                        fuzz.ratio(r['标准名2'], b_val)+   \
                        fuzz.ratio(r['标准名3'], c_val)+   \
                        fuzz.ratio(r['标准名4'], d_val)   )/4
            
            r['fuzz_ratio'] = fuzz_value
            r['index_id'] = index
            temp_df = temp_df.append(r)

            # 如果当前的fuzz_ratio大于已知的最大值，则更新最大值和对应的行
            if fuzz_value > max_ratio:
                max_ratio = fuzz_value
                max_ratio_row = r
        if isStandardProductionCompanyExists ==False:
            matching_Manufacturer_rows = df1[(df1['标准学科'] == row['学科']) ]
            matching_Manufacturer_rows = matching_Manufacturer_rows.assign(标准学科匹配=1)
            matching_Manufacturer_rows = matching_Manufacturer_rows.assign(index_id=index)
            temp_df = temp_df.append(matching_Manufacturer_rows)


        # 每次循环结束后，对temp_df按照'fuzz_ratio'降序排序，然后添加到matched_df中
        if 'fuzz_ratio' in temp_df.columns:
            temp_df = temp_df.sort_values(by='fuzz_ratio', ascending=False)
            matched_df = matched_df.append(temp_df)

        # 将每个min1对应的fuzz_ratio最高的行添加到highest_ratio_df中
        if max_ratio_row is not None:
            highest_ratio_df = highest_ratio_df.append(max_ratio_row)
    return matched_df,highest_ratio_df
# 打印最终得到的匹配结果
# print(matched_df)
# print(highest_ratio_df)